In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
leis = pd.read_json('leis.json')
leis.drop(['documento'], inplace=True, axis=1)
print(leis.info())
print(leis.nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6033 entries, 0 to 6032
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   titulo     6033 non-null   object
 1   categoria  6033 non-null   object
 2   resumo     6033 non-null   object
 3   texto      6033 non-null   object
dtypes: object(4)
memory usage: 188.7+ KB
None
titulo       6033
categoria       8
resumo       4961
texto        6029
dtype: int64


In [3]:
leis

,titulo,categoria,resumo,texto
0,"DECRETO Nº 8854, de 28 de fevereiro de 2013.",Decretos,DELEGA COMPETÊNCIA À SECRETARIA MUNICIPAL DE P...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
1,"DECRETO Nº 8849, de 25 de fevereiro de 2013.",Decretos,ABRE CRÉDITO SUPLEMENTAR AO ORÇAMENTO DO MUNIC...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
2,"DECRETO Nº 8853, de 27 de fevereiro de 2013.",Decretos,NOMEIA MEMBROS DO CONSELHO MUNICIPAL DE DESENV...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
3,"DECRETO Nº 8967, de 17 de julho de 2013",Decretos,ALTERA O QUADRO DE DETALHAMENTO DE DESPESA DO ...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
4,"DECRETO Nº 8982, de 30 de julho de 2013",Decretos,AUTORIZA O FUNCIONAMENTO DE ESTABELECIMENTOS C...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
...,...,...,...,...
6028,RESOLUÇÃO Nº 125/1980,Resoluções,DISPÕE SOBRE A CONCESSÃO DE TÍTULO DE CIDADÃO ...,Faço saber que a Câmara Municipal aprovou e eu...
6029,RESOLUÇÃO Nº 403/2003,Resoluções,AUTORIZA A MESA DIRETIVA DO PODER LEGISLATIVO ...,"A CÂMARA MUNICIPAL DE FEIRA DE SANTANA, Estado..."
6030,RESOLUÇÃO Nº 492/2014,Resoluções,INSTITUI A SEGUNDA SEMANA DO MÊS DE AGOSTO EM ...,"A CÂMARA MUNICIPAL DE FEIRA DE SANTANA, Estado..."
6031,RESOLUÇÃO Nº 382/2001,Resoluções,CRIA A MEDALHA VEREADOR DIVAL FIGUEIREDO MACHA...,"A CÂMARA MUNICIPAL DE FEIRA DE SANTANA, Estado..."


In [4]:
# Exemplo de texto de lei
leis.loc[len(leis)-1, 'texto']

'A CÂMARA MUNICIPAL DE FEIRA DE SANTANA, Estado da Bahia, na conformidade do artigo 70, Inciso V, da Lei Municipal nº37, de 05 de Abril de 1990 e, artigos 287, § 2º e, 420, do Regimento Interno, promulga a seguinte Resolução:\n\nArt. 1ºDê-se aos dispositivos abaixo mencionados, da Resolução nº393/2002 - Regimento Interno, as seguintes redações:\n\n"Art. 7º A Mesa Diretora da Câmara compor-se-á do Presidente, Primeiro e Segundo Secretários, com mandato de 02 ( dois ) anos, admitida a recondução para a eleição subsequente.\n\n§ 4º Se, hora regimental, não estiver presente o Presidente, abrirá os trabalhos o Vice-Presidente ou, na falta deste, o Primeiro ou Segundo Secretários, na sequência, ou ainda, caso estes não estejam presentes, o Vereador mais votado nas eleições municipais."\n\n"Art. 33 Compete, privativamente, ao Vice-Presidente:"\n\n"Art. 36 ...\n\nI - ...\n\ne) acompanhar e supervisionar a Ata da Sessão, proceder a sua leitura e assiná-la depois do Presidente e do Vice-Presiden

# Semelhança de documentos

In [6]:
from scripts.parsers import limpa_texto
leis['texto_limpo'] = leis['texto'].apply(limpa_texto)

In [7]:
# Constrói matriz de documentos
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(leis['texto_limpo'])
X

<6033x28875 sparse matrix of type '<class 'numpy.int64'>'
	with 534596 stored elements in Compressed Sparse Row format>

In [8]:
def print_lei_mais_sem(orig_idx : int, sem_idx: int):
    print(f'- - - LEI COMPARADA {orig_idx}: - - -\n\n')
    print(leis.loc[orig_idx, 'texto'])
    print(f'\n\n- - - LEI MAIS SEMELHANTE {sem_idx} - - -\n\n')
    print(leis.loc[sem_idx, 'texto'])

In [9]:
# Calcular similaridade usando TF-IDF agora
# Comparar com a mesma lei original anterior
transformer = TfidfTransformer()
X_tfidf = transformer.fit_transform(X)

X_tfidf

<6033x28875 sparse matrix of type '<class 'numpy.float64'>'
	with 534596 stored elements in Compressed Sparse Row format>

# Buscas por texto

Já vimos que TF-IDF encontra Leis bastante similares entre si. Será que conseguimos também encontrar leis similares a uma query (consulta)? 

Primeiro temos que limpar o texto da consulta utilizando o mesmo método de limpeza das leis. Depois, transformar a consulta pra utilizar o IDF do modelo treinado. Finalmente, calcular a similaridade desta consulta pra todas as leis na base e retornar a mais próxima ou mais próximas.

In [14]:
consulta = ['winterianus']
consulta[0] = limpa_texto(consulta[0])
consulta = vectorizer.transform(consulta)
consulta = transformer.transform(consulta)

In [15]:
# Pra achar os similares, computar a similaridade do cosseno
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

mais_proximos = cosine_similarity(consulta, X_tfidf)
mais_proximos_idx = np.argsort(mais_proximos)
for i in range(1,5):
    idx = mais_proximos_idx[0, -i]
    print(leis.loc[idx, 'texto'])
    print('\n---proximo resultado---\n')

 Diego de Oliveira Silva Azevedo
f) Sônia Maria Tavares Rodrigues de Melo Mascarenhas
g) Patrícia Nascimento de Jesus".

Art. 2ºO § 1º, do art. 2º, do Decreto nº8.834, de 10 de janeiro de 2013, passa a viger com a seguinte redação:

"§ 1º A Presidência da Comissão será exercida pela servidora ADRIANA ESTELA BARBOSA ASSIS, e nas suas ausências ou impedimentos pela servidora CAROLINE SUZART COTIAS FREITAS, ou outro qualquer titular, desde que designado pela Presidência."

Art. 3ºO § 2º, do art. 2º, do Decreto nº8.834, de 10 de janeiro de 2013, passa a ter a seguinte redação:

"2º Nos Pregões Eletrônicos, bem como nos Pregões Presenciais, a titularidade ficará a cargo da Pregoeira Municipal ADRIANA ESTELA BARBOSA ASSIS, e nas suas ausências ou impedimentos pelos membros titulares CADMIEL MASCARENHAS PEREIRA, CAROLINE SUZART COTIAS FREITAS, GISELLE CRISTINE DE GÓES COSTA BOTELHO, ou AIDIL PINHEIRO DO NASCIMENTO, podendo requisitar também os demais membros Titulares, Suplentes e Assistentes

Tcharam! Feito o buscador!

Existem limitações. A sequência e composição das palavras é uma delas, por exemplo. Não adianta buscar pelo nome - sobrenome de uma pessoa que ele vai retornar resultados onde algum destes termos sejam mais frequentes. Não existe as aspas do Google pra dizer "busque por este termo todo junto".

Por exemplo, se eu buscar Elydio aparece a primeira Lei conferindo cidadania à Elydio Azevedo Lopes. Perfeito. Mas se eu buscar Azevedo Lopes, o primeiro resultado sequer tem Azevedo, mas o nome Lopes aparece mais de uma vez.

Uma das formas de contornar essa dificuldade é usar bigramas ou ngramas de palavras maiores. Bigramas parecem fazer sentido pra nome de bairros, ruas e pessoas, já que são compostos de 2 nomes.


## Outras opções
### Indexar
Há outras formas de indexar os documentos e de recuperar, também simples. Uma outra forma de indexar, por exemplo, é fazer um vetor pra cada palavra contando as palavras vizinhas. E depois, o vetor do documento seria a soma dos vetores das palavras. É uma forma interessante porque pode gerar visualizações interessantes entre a similaridade das palavras. Por exemplo, no corpus das Leis Municipais, a quais palavras EDUCAÇÃO mais se assemelha? Ou SAÚDE? Etc.

Outra forma é contar n-gramas - por exemplo, bi-gramas: duas palavras juntas formando um token. Dessa forma, você possui uma matriz maior e de certa forma uma relação entre a sequencialidade das palavras, que pode ser útil pra nomes de pessoas e bairros, como citado acima.

### Recuperar
Outra forma de recuperar é por local sensitive hashing. Divide em vários planos múltiplas vezes e retorna os resultados que estão na mesma região da query. No entanto, o corpus não é grande o suficiente pra precisar essa estratégia, que é mais pra grandes corpora. O método acima (calcular a simlaridade cosseno e retornar os maiores valores) é rápido o suficiente pra parecer instantâneo. Talvez com uma demanda mais alta pelo servidor venha a necessidade de aumentar a velocidade da busca, porém por enquanto não é o caso.

### Avaliação
Com múltiplas formas de indexar e recuperar vem o dilema: como avaliar se uma é melhor que a outra? Repetir o processo acima pra todas as opções? Isto é, mostrar N melhores resultados e comparar manualmente? Ou colocar labels em algumas leis? Ex: essa lei trata disso, com tais entidades. Checar formas de avaliação. Se tivesse em produção, podia avaliar por CTR por ex, mas não é o caso